# Convolutional Neural Network in Keras

Bulding a Convolutional Neural Network to classify Zalando's MNIST dataset.

#### Set seed for reproducibility

In [ ]:
import numpy as np
np.random.seed(42)

#### Load dependencies

In [ ]:
!pip install --upgrade git+https://www.github.com/ekholabs/keras-contrib.git@improve-sinerelu

In [ ]:
import os
import keras
from keras.datasets import fashion_mnist
from keras.models import Sequential
from keras.layers import Layer, Activation, Dense, Dropout, Conv2D, MaxPooling2D, Flatten, LeakyReLU, BatchNormalization
from keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping

from keras import backend as K
from keras.utils.generic_utils import get_custom_objects
from keras_contrib.layers.advanced_activations import SineReLU

#### Load data

In [ ]:
(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()

#### Preprocess data
Flatten and normalise input data.

In [ ]:
X_train = X_train.reshape(-1, 28, 28, 1)
X_test = X_test.reshape(-1, 28, 28, 1)

X_train = X_train.astype("float32")/255.
X_test = X_test.astype("float32")/255.

In [ ]:
# One-hot encoded categories
n_classes = 10
y_train = keras.utils.to_categorical(y_train, n_classes)
y_test = keras.utils.to_categorical(y_test, n_classes)

#### Design Neural Network architecture

In [ ]:
epsilon_cnn, epsilon_dense = 0.025, 0.006

model = Sequential()

model.add(Conv2D(32, 7, padding = 'same', input_shape = (28, 28, 1)))
# model.add(SineReLU(epsilon_cnn))
# model.add(LeakyReLU(alpha=0.01))
model.add(Activation('relu'))

model.add(Conv2D(32, 7, padding = 'same'))
# model.add(SineReLU(epsilon_cnn))
# model.add(LeakyReLU(alpha=0.01))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(0.20))

model.add(Conv2D(64, 3, padding = 'same'))
# model.add(SineReLU(epsilon_cnn))
# model.add(LeakyReLU(alpha=0.01))
model.add(Activation('relu'))

model.add(Conv2D(64, 3, padding = 'same'))
# model.add(SineReLU(epsilon_cnn))
# model.add(LeakyReLU(alpha=0.01))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(0.30))

model.add(Conv2D(128, 2, padding = 'same'))
# model.add(SineReLU(epsilon_cnn))
# model.add(LeakyReLU(alpha=0.01))
model.add(Activation('relu'))

model.add(Conv2D(128, 2, padding = 'same'))
# model.add(SineReLU(epsilon_cnn))
# model.add(LeakyReLU(alpha=0.01))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(0.40))

model.add(Flatten())
model.add(Dense(512))
# model.add(SineReLU(epsilon_dense))
# model.add(LeakyReLU(alpha=0.01))
model.add(Activation('relu'))
model.add(Dropout(0.50))

model.add(Dense(10, activation = "softmax"))

model.summary()

#### Callbacks

In [ ]:
modelCheckpoint = ModelCheckpoint(monitor='val_acc', filepath='model_output/weights-cnn-mnist.hdf5',
                                               save_best_only=True, mode='max')
earlyStopping = EarlyStopping(monitor='val_acc', mode='max', patience=30)


if not os.path.exists('model_output'):
    os.makedirs('model_output')

tensorboard = TensorBoard("../new_logs/convnet-mnist-relu-VIII")

#### Configure model

In [ ]:
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

#### Train!

In [ ]:
history = model.fit(X_train, y_train, batch_size = 128, epochs = 40, verbose = 1,
          validation_split = 0.1, callbacks=[modelCheckpoint, earlyStopping, tensorboard])

#### Test Predictions

In [ ]:
saved_model = keras.models.load_model('model_output/weights-cnn-mnist.hdf5')
predictions = saved_model.predict_classes(X_test, verbose = 2)
np.std(history.history['loss'])

#### Test Final Accuracy

In [ ]:
final_loss, final_acc = saved_model.evaluate(X_test, y_test, verbose = 1)
print("Final loss: {0:.4f}, final accuracy: {1:.4f}".format(final_loss, final_acc))
final_loss, final_acc = model.evaluate(X_test, y_test, verbose = 1)
print("Final loss: {0:.4f}, final accuracy: {1:.4f}".format(final_loss, final_acc))